# 2.1.2 Building blocks for programming preconditioners

In [1]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
%gui tk
from ngsolve.la import EigenValues_Preconditioner
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

In [2]:
fes = H1(mesh, order=3, dirichlet="left|bottom")
u, v = fes.TnT()
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v) + u*v)
a.Assemble()
f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()
gfu = GridFunction(fes)

## Create a Jacobi-preconditioner

Let  $A=$ `a.mat` be the assembled matrix, which can be decomposed based on `FreeDofs` ($F$) the remainder ($D$), as in $\S$[1.3](../unit-1.3-dirichlet/dirichlet.ipynb),

$$
A = \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right). 
$$

Then the matrix form of the **point Jacobi preconditioner** is

$$
J = \left( \begin{array}{cc} \text{diag}(A_{FF})^{-1} & 0  \\ 0 & I  \end{array} \right),
$$

which can be obtained in NGSolve using `CreateSmoother`:

In [3]:
preJpoint = a.mat.CreateSmoother(fes.FreeDofs())

NGSolve also gives us a facility to quickly check an estimate of the condition number of the preconditioned matrix by applying the Lanczos algorithm on the preconditioned system.


In [4]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preJpoint)
lams

 0.0146975
 0.0567568
 0.0954973
 0.112599
 0.149013
 0.198016
 0.266185
 0.354506
 0.452415
 0.535142
 0.63906
 0.751009
 0.836039
 0.932691
 1.04077
 1.16473
 1.24579
 1.36685
 1.47433
 1.58784
 1.70789
 1.83511
 1.92562
 2.02092
 2.12651
 2.20727
 2.27824
 2.38882
 2.44463
 2.47644
 2.51031
 2.54019
  2.8097

An estimate of the condition number

$$
\kappa = \frac{\lambda_{\text{max}} }{ \lambda_{\text{min}}}
$$

is therefore given as follows:

In [5]:
max(lams)/min(lams)

191.168278526202

One might wonder if we have gained anything by this point Jacobi preconditioning. What if we did not precondition at all?

Not preconditioning is the same as preconditioning by the identity operator on $F$-dofs. One way to realize this identity operator in NGSolve is through the projection into the space of free dofs (i.e., the space spanned by the shape functions corresponding to free dofs). NGSolve provides

```
Projector(mask, range)   # mask: bit array, range: bool
```

which projects into the space spanned by the shape functions of the degrees of freedom marked as range in mask.

In [6]:
preI = Projector(mask=fes.FreeDofs(), range=True)

*Note*  that another way to obtain the identity matrix in NGSolve is    
```
IdentityMatrix(fes.ndof, complex=False).
```

In [7]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
max(lams)/min(lams)

1585.7752877045514

Clearly the point Jacobi preconditioner has reduced the condition number. 

We can use preconditioners within iterative solvers provided by NGSolve's `solvers` module (which has `MinRes`, `QMR` etc.) Here is an illustration of its use within CG, or the **conjugate gradient** solver: 

In [8]:
solvers.CG(mat=a.mat, pre=preJpoint, rhs=f.vec, sol=gfu.vec)
Draw(gfu)

it =  0  err =  0.05522230048185231
it =  1  err =  0.09382744174195908
it =  2  err =  0.10587866002039004
it =  3  err =  0.09033871153608707
it =  4  err =  0.10086392272140011
it =  5  err =  0.08517119355086304
it =  6  err =  0.08906575044224518
it =  7  err =  0.07684062913929467
it =  8  err =  0.056047175061895105
it =  9  err =  0.04472071103380234
it =  10  err =  0.03419741252004591
it =  11  err =  0.02682484049645099
it =  12  err =  0.022322983522404753
it =  13  err =  0.020056199326844725
it =  14  err =  0.014372559087884462
it =  15  err =  0.010772925966275729
it =  16  err =  0.009633149024762148
it =  17  err =  0.0073015321836821315
it =  18  err =  0.004811737390491996
it =  19  err =  0.0032534005521665317
it =  20  err =  0.001902580694234345
it =  21  err =  0.0012857479602116576
it =  22  err =  0.0010097069801732902
it =  23  err =  0.0006971063184209084
it =  24  err =  0.0004415360534270248
it =  25  err =  0.00031313626798158353
it =  26  err =  0.000245

##  Gauss-Seidel smoothing

The *same* point Jacobi smoother object can also used to perform **point Gauss-Seidel** smoothing. One step of the classical Gauss-Seidel iteration is realized by the method `preJpoint.Smooth()`. Its well known that this iteration converges for matrices like $A$. Below we show how to use it as a linear iterative solver. 


In [9]:
gfu.vec[:] = 0
res = f.vec.CreateVector()              # residual 
projres = f.vec.CreateVector()          # residual projected to freedofs
proj = Projector(fes.FreeDofs(), True)

for i in range(500):
    preJpoint.Smooth(gfu.vec, f.vec)    # one step of point Gauss-Seidel
    res.data = f.vec - a.mat*gfu.vec      
    projres.data = proj * res
    print ("it#", i, ", res =", Norm(projres))
Draw (gfu)

it# 0 , res = 0.08192679624402945
it# 1 , res = 0.07726762555864525
it# 2 , res = 0.07377040495808554
it# 3 , res = 0.07062220895606912
it# 4 , res = 0.06777582421551728
it# 5 , res = 0.06516770753241384
it# 6 , res = 0.06275317789329397
it# 7 , res = 0.06050089690216452
it# 8 , res = 0.058387526692409986
it# 9 , res = 0.056394947347621995
it# 10 , res = 0.0545086993995133
it# 11 , res = 0.05271702122673418
it# 12 , res = 0.05101020777328185
it# 13 , res = 0.04938015983450581
it# 14 , res = 0.04782005439140267
it# 15 , res = 0.04632409596387132
it# 16 , res = 0.04488732463071546
it# 17 , res = 0.04350546518198168
it# 18 , res = 0.042174807039847335
it# 19 , res = 0.04089210774233109
it# 20 , res = 0.0396545147959703
it# 21 , res = 0.03845950204060288
it# 22 , res = 0.03730481759400025
it# 23 , res = 0.0361884411063836
it# 24 , res = 0.03510854854333058
it# 25 , res = 0.03406348308437501
it# 26 , res = 0.033051731008142335
it# 27 , res = 0.032071901655931344
it# 28 , res = 0.03112271073

it# 302 , res = 9.20142638007839e-06
it# 303 , res = 8.93264385864601e-06
it# 304 , res = 8.671712733319411e-06
it# 305 , res = 8.418403657416411e-06
it# 306 , res = 8.172493983483656e-06
it# 307 , res = 7.933767567766574e-06
it# 308 , res = 7.702014580399288e-06
it# 309 , res = 7.477031320963506e-06
it# 310 , res = 7.25862003899917e-06
it# 311 , res = 7.046588760873389e-06
it# 312 , res = 6.84075112049103e-06
it# 313 , res = 6.640926195685272e-06
it# 314 , res = 6.446938349203836e-06
it# 315 , res = 6.25861707457476e-06
it# 316 , res = 6.075796845556455e-06
it# 317 , res = 5.898316971400937e-06
it# 318 , res = 5.72602145524725e-06
it# 319 , res = 5.5587588569724775e-06
it# 320 , res = 5.396382160124041e-06
it# 321 , res = 5.2387486429536435e-06
it# 322 , res = 5.085719752475799e-06
it# 323 , res = 4.9371609832884464e-06
it# 324 , res = 4.792941758633401e-06
it# 325 , res = 4.652935316508108e-06
it# 326 , res = 4.5170185971830715e-06
it# 327 , res = 4.385072136057004e-06
it# 328 , res 

## Implement a forward-backward GS preconditioner

The *same* point Jacobi smoother object is also able to perform a Gauss-Seidel iteration after reversing the ordering of the points, i.e., a **backward** Gauss-Seidel sweep. One can combine the forward and backward sweeps to construct a symmetric preconditioner, often called the **symmetrized Gauss-Seidel preconditioner**. This offers a good illustration of how to construct NGSolve preconditioners from within python. 

In [10]:
class SymmetricGS(BaseMatrix):
    def __init__ (self, smoother):
        super(SymmetricGS, self).__init__()
        self.smoother = smoother
    def Mult (self, x, y):
        y[:] = 0.0
        self.smoother.Smooth(y, x)
        self.smoother.SmoothBack(y,x)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

In [11]:
preGS = SymmetricGS(preJpoint)
solvers.CG(mat=a.mat, pre=preGS, rhs=f.vec, sol=gfu.vec)
Draw (gfu)

it =  0  err =  0.09421429683611413
it =  1  err =  0.12242092674801901
it =  2  err =  0.08627911974342342
it =  3  err =  0.054294636877542546
it =  4  err =  0.026834953641369612
it =  5  err =  0.014442946534311274
it =  6  err =  0.006828162088958496
it =  7  err =  0.002947538932807923
it =  8  err =  0.0009884874866274685
it =  9  err =  0.00037122112200935675
it =  10  err =  0.00011770383684169319
it =  11  err =  4.7294009691925314e-05
it =  12  err =  1.795956404345207e-05
it =  13  err =  7.73981616121209e-06
it =  14  err =  3.688537792536532e-06
it =  15  err =  1.7853513785615997e-06
it =  16  err =  6.793337817618327e-07
it =  17  err =  3.267271476258251e-07
it =  18  err =  1.0509766947753396e-07
it =  19  err =  5.2632976611271436e-08
it =  20  err =  1.7518834053967413e-08
it =  21  err =  7.1749421044469985e-09
it =  22  err =  2.5718606236788454e-09
it =  23  err =  7.674623733376304e-10
it =  24  err =  3.2317029440816587e-10
it =  25  err =  7.901568725512893e-1

In [12]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preGS)
max(lams)/min(lams)

20.165532778155335

Note that the condition number now is better than that of the system preconditioned by point Jacobi.

## A Block Jacobi preconditioner

The point Jacobi preconditioner is based on inverses of 1 x 1 diagonal blocks.  Condition numbers can be improved by using larger blocks. It is possible to group dofs into blocks within python and construct an NGSolve preconditioner based on the blocks.

Here is an example that constructs vertex-based blocks.

In [13]:
blocks = []
freedofs = fes.FreeDofs()
for v in mesh.vertices:
    vdofs = set()
    for el in mesh[v].elements:
        vdofs |= set(d for d in fes.GetDofNrs(el) if freedofs[d])
    blocks.append (vdofs)
print (blocks)

[{866, 158, 159}, {13, 206, 207, 48, 145, 144, 882, 142, 143, 210, 211, 884}, {256, 257, 2, 901, 146, 147, 148, 21, 22, 149}, {65, 314, 919, 315, 917, 310, 150, 151, 154, 155, 311, 30}, {160, 161, 866, 867, 164, 165, 935, 40, 361, 360, 158, 159}, {869, 160, 161, 867, 164, 165, 868, 166, 40, 167, 41, 170, 171, 362, 363}, {868, 166, 167, 870, 41, 170, 171, 42, 172, 173, 871, 176, 177, 368, 369}, {375, 870, 872, 873, 42, 43, 172, 173, 176, 177, 178, 179, 182, 183, 374}, {380, 872, 874, 43, 44, 875, 178, 179, 381, 182, 183, 184, 185, 188, 189}, {194, 195, 386, 387, 874, 44, 876, 45, 877, 184, 185, 188, 189, 190, 191}, {194, 195, 196, 197, 200, 201, 392, 393, 876, 45, 46, 878, 879, 190, 191}, {196, 197, 200, 201, 202, 203, 204, 205, 398, 399, 878, 46, 47, 880, 881}, {202, 203, 204, 205, 206, 207, 144, 145, 404, 405, 47, 880, 48, 882, 883}, {13, 142, 143, 144, 145, 210, 211, 14, 208, 209, 212, 213, 217, 216, 410, 411, 48, 49, 884, 885, 886}, {13, 14, 15, 208, 209, 212, 213, 214, 215, 216, 21

`CreateBlockSmoother` can now take these blocks and construct a block Jacobi preconditioner.

In [14]:
blockjac = a.mat.CreateBlockSmoother(blocks)

lams = EigenValues_Preconditioner(mat=a.mat, pre=blockjac)
max(lams)/min(lams)

34.84033814053211

Multiplicative smoothers and its symmetrized version often yield better condition numbers in practice. We can apply the same code we wrote above for symmetrization (`SymmetricGS`) to the block smoother:

In [15]:
blockgs = SymmetricGS(blockjac)

lams = EigenValues_Preconditioner(mat=a.mat, pre=blockgs)
max(lams)/min(lams)

2.982606144898276

## Add a coarse grid correction

Dependence of the condition number on degrees of freedom can often be reduced by preconditioners that appropriately use a coarse grid correction.  It is also possible to experiment with coarse grid corrections using NGSolve's python interface. We now show how to precondition with a coarse grid correction made using the lowest order subspace of `fes`.

In the example below, note that we use `fes.GetDofNrs` again. Previously we used it with argument `el` of type `ElementId`, while now we use it with an argument `v` of type `MeshNode`.

In [16]:
vertexdofs = BitArray(fes.ndof)
vertexdofs[:] = False

for v in mesh.vertices:
    for d in fes.GetDofNrs(v):
        vertexdofs[d] = True
        
vertexdofs &= fes.FreeDofs()

print(vertexdofs)    # bit array, printed 50 chars/line

0: 00100000000001111111111111111110000000001111111111
50: 11111111111111111111111111111111111111111111111111
100: 11111111111111111111111111111111111100000000000000
150: 00000000000000000000000000000000000000000000000000
200: 00000000000000000000000000000000000000000000000000
250: 00000000000000000000000000000000000000000000000000
300: 00000000000000000000000000000000000000000000000000
350: 00000000000000000000000000000000000000000000000000
400: 00000000000000000000000000000000000000000000000000
450: 00000000000000000000000000000000000000000000000000
500: 00000000000000000000000000000000000000000000000000
550: 00000000000000000000000000000000000000000000000000
600: 00000000000000000000000000000000000000000000000000
650: 00000000000000000000000000000000000000000000000000
700: 00000000000000000000000000000000000000000000000000
750: 00000000000000000000000000000000000000000000000000
800: 00000000000000000000000000000000000000000000000000
850: 0000000000000000000000000000000000000000000000

Thus we have made a mask `vertexdofs` which reveals all free dofs associated to vertices. If these are labeled $c$ (and the remainder is labeled $f$), then the matrix $A$ can partitioned into 

$$
A = \left( \begin{array}{cc} A_{cc} & A_{cf} \\ A_{fc} & A_{ff} \end{array} \right). 
$$

The matrix `coarsepre` below represents

$$
\left( \begin{array}{cc} A_{cc}^{-1} & 0 \\ 0 & 0 \end{array} \right). 
$$

In [17]:
coarsepre = a.mat.Inverse(vertexdofs)

This matrix can be used for coarse grid correction. 

*Pitfall!*  Note that `coarsepre` is not appropriate as a preconditioner by itself as it has a large null space. You might get the wrong idea from the results of a Lanczos eigenvalue estimation:

In [18]:
EigenValues_Preconditioner(mat=a.mat, pre=coarsepre)

       1

But this result only gives the Laczos eigenvalue estimates on the *range* of the preconditioner. The preconditioned operator in this case is simply 

$$
\left( \begin{array}{cc} A_{cc}^{-1} & 0 \\ 0 & 0 \end{array} \right)
\left( \begin{array}{cc} A_{cc} & A_{cf} \\ A_{fc} & A_{ff} \end{array} \right)
 = 
 \left( \begin{array}{cc} I  & A_{cc}^{-1} A_{cf} \\ 0 & 0  \end{array} \right),
$$

which is a projection into the $c$-dofs. Hence its no surprise that Lanczos estimated the eigenvalues of this operator (on its range) to be just one. But this does not imply that the condition number of this preconditioned system is nice.

One well-known and correct way to combine the coarse grid correction with one of the previous smoothers is to combine them additively,  to get an **additive two-grid preconditioner** as follows.

In [19]:
twogrid = coarsepre + blockgs 

This addition of two operators (of type `BaseMatrix`) results in another operator, which is stored as an expression, to be evaluated only when needed.  The 2-grid preconditioner has a very good condition number.

In [20]:
EigenValues_Preconditioner(mat=a.mat, pre=twogrid)

 0.993081
 0.997768
 0.999961
 1.34206
 1.80314
 1.83857
 1.96023
  1.9828
 1.99987